In [1]:
import json
import os

import matplotlib.pyplot as plt
import pandas as pd
import utils
import seaborn as sn

plt.style.use("ggplot")

# Explore ODM results

This notebook compares metrics from the ODM parameter search to output from a reference Pix4D dataset.

# User input

In [2]:
# Version of parameter combo file to use
param_version = 0

## 2. Read metrics from ODM results

In [3]:
# Metrics of interest (from stats.json)
# Keys are separate by '-' to define the path in the nested JSON, values are the names to use
# in the summary dataframe
attrib_dict = {
    "processing_statistics-area": "area_m2",
    "odm_processing_statistics-total_time": "processing_time_s",
    "odm_processing_statistics-average_gsd": "mean_gsd_cm",
    "features_statistics-reconstructed_features-median": "median_recon_features",
    "reconstruction_statistics-initial_points_count": "initial_points",
    "reconstruction_statistics-reconstructed_points_count": "reconstructed_points",
    "gps_errors-ce90": "horiz_acc_ce90_m",
    "gps_errors-le90": "vert_acc_le90_m",
}

In [4]:
# Identify result dirs
res_dir = f"/home/notebook/shared-seabee-ns9879k/niva-tidy/2025/odm_tuning/params_v{param_version:02d}"
dir_list = [f for f in os.listdir(res_dir) if os.path.isdir(os.path.join(res_dir, f))]

# Build dict for results
res_dict = {i: [] for i in attrib_dict.values()}
res_dict["run_id"] = []

for dir_name in dir_list:
    run_id = int(dir_name.split("-")[-1])

    # Read ODM stats
    stats_path = os.path.join(res_dir, dir_name, "report", "stats.json")
    if os.path.exists(stats_path):
        res_dict["run_id"].append(run_id)
        with open(stats_path, "r") as file:
            stats_dict = json.load(file)

        # Get data of interest
        for json_path, attrib_name in attrib_dict.items():
            val = utils.get_attribute_by_path(stats_dict, json_path)
            res_dict[attrib_name].append(val)

# Build dataframe
df = (
    pd.DataFrame(res_dict)
    .set_index("run_id")
    .sort_index()
    .reset_index()
)
df

,run_id,area_m2,processing_time_s,mean_gsd_cm,median_recon_features,initial_points,reconstructed_points,horiz_acc_ce90_m,vert_acc_le90_m
0,0,39310.731150,11200.735072,0.796630,4063,427802,418723,0.046537,0.026021
1,1,39301.960685,14213.449463,0.798107,4105,429656,422936,0.091807,0.046585
2,2,39321.780487,14999.535662,0.797135,3795,380693,379607,0.098632,0.061453


In [5]:
# stats_dict

In [6]:
# sn.pairplot(df)#, hue='run_id', diag_kind='hist')